In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/train.csv", nrows=400000)
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_test = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/test.csv")
df_test.head()

In [ ]:
df_train.info()
df_test.info()

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
df_train.drop(df_train[df_train["passenger_count"] > 5].index, axis=0, inplace=True)
df_train.drop(df_train[df_train["passenger_count"] == 0].index, axis=0, inplace=True)

In [ ]:
df_train.drop(df_train[df_train["fare_amount"] < 3].index, axis=0, inplace=True)
df_train.drop(df_train[df_train["fare_amount"] > 200].index, axis=0, inplace=True)

In [ ]:
df_train["key"] = pd.to_numeric(df_train["key"].index, downcast="float")

In [ ]:
df_test["key"] = pd.to_numeric(df_test["key"].index, downcast="float")

In [ ]:
df_train.dropna(inplace=True)

df_train.drop(df_train.index[(df_train.pickup_longitude < -75.0)|
                             (df_train.pickup_longitude > -72.0)|
                             (df_train.pickup_latitude < 40.0)|
                             (df_train.pickup_latitude > 42.0)], inplace=True)

df_train.drop(df_train.index[(df_train.dropoff_longitude < -75.0)|
                             (df_train.dropoff_longitude > -72.0)|
                             (df_train.dropoff_latitude < 40.0)|
                             (df_train.dropoff_latitude > 42.0)], inplace=True)


In [ ]:
df_train.describe()

In [ ]:
from sklearn.model_selection import train_test_split

X =  df_train.drop(["fare_amount", "pickup_datetime"], axis=1)

y = df_train["fare_amount"]

X_test = df_test.drop(["pickup_datetime"], axis=1)
                       
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=12)

In [ ]:
train_x.dtypes

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(train_x, train_y)
dvalid = xgb.DMatrix(data=valid_x, label=valid_y)
dtest = xgb.DMatrix(X_test)
watchlist = [(dtrain, "train"), (dvalid, "valid")]

params = {"min_child_weight": 1,
          "learning_rate":0.005,
          "colsample_bytree":0.7,
          "max_depth":10,
          "subsample":0.7,
          "n_jobs":-1,
          "booster":"gbtree",
          "eval_metric":"rmse"
         }

model = xgb.train(params, dtrain,700, watchlist, early_stopping_rounds=100, maximize=False, verbose_eval=50)

In [ ]:
prediction = model.predict(dtest)

In [ ]:
submission = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv")

In [ ]:
output = pd.DataFrame({"key": submission["key"], "fare_amount": prediction})
output

In [ ]:
output.to_csv("my_submission", index=False)